In [12]:
# imports

import os
from openai import OpenAI
import gradio as gr

In [ ]:

openai = OpenAI(
    api_key= 'key',
    base_url='https://api.openai.com/v1'
)
MODEL = 'gpt-4o-mini'

Please read this! A change from the video:¶
In the video, I explain how we now need to write a function called:

chat(message, history)

Which expects to receive history in a particular format, which we need to map to the OpenAI format before we call OpenAI:

[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
But Gradio has been upgraded! Now it will pass in history in the exact OpenAI format, perfect for us to send straight to OpenAI.

So our work just got easier!

We will write a function chat(message, history) where:
message is the prompt to use
history is the past conversation, in OpenAI format

We will combine the system message, history and latest message, then call OpenAI.

In [14]:
system_message = "You are a helpful assistant"

In [15]:
# Simpler than in my video - we can easily create this function that calls OpenAI
# It's now just 1 line of code to prepare the input to OpenAI!

# Student Octavio O. has pointed out that this isn't quite as straightforward for Claude -
# see the excellent contribution in community-contributions "Gradio_issue_with_Claude" that handles Claude.

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [11]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


History is:
[]
And messages is:
[{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'Hi , what is longchain, give in 40 words'}]
History is:
[]
And messages is:
[{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'Hi , what is longchain, give only the application of it'}]
History is:
[{'role': 'user', 'metadata': None, 'content': 'Hi , what is longchain, give only the application of it', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Longchain is a blockchain framework primarily designed for enabling decentralized finance (DeFi) applications, facilitating cross-chain interactions, and providing a platform for the development of scalable and efficient distributed applications. Some of the applications of Longchain include:\n\n1. **Decentralized Exchanges (DEXs)**: Allow users to trade cryptocurrencies directly without intermediaries.\n2. **Liquidity Pools**: Users can provide liquidity f

In [16]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

In [17]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [18]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [26]:
# Fixed a bug in this function brilliantly identified by student Gabor M.!
# I've also improved the structure of this function

def chat(message, history):

    relevant_system_message = system_message
    if 'belt' in message:
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [27]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
